In [1]:
%matplotlib inline


Autograd: 자동 미분
===================================

PyTorch의 모든 신경망의 중심에는 ``autograd`` 패키지가 있습니다.
먼저 이것을 가볍게 살펴본 뒤, 첫번째 신경망을 학습시켜보겠습니다.

``autograd`` 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공합니다.
이는 실행-기반-정의(define-by-run) 프레임워크로, 이는 코드를 어떻게 작성하여
실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다
달라집니다.

더 간단한 용어로 몇 가지 예를 살펴보겠습니다.

Tensor
--------

패키지의 중심에는 ``torch.Tensor`` 클래스가 있습니다. 만약 ``.requires_grad``
속성을 ``True`` 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기
시작합니다. 계산이 완료된 후 ``.backward()`` 를 호출하여 모든 변화도(gradient)를
자동으로 계산할 수 있습니다. 이 Tensor의 변화도는 ``.grad`` 속성에 누적됩니다.

Tensor가 기록을 추적하는 것을 중단하게 하려면, ``.detach()`` 를 호출하여 연산
기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지할 수 있습니다.

기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하기 위해, 코드 블럭을
``with torch.no_grad():`` 로 감쌀 수 있습니다. 이는 특히 변화도(gradient)는
필요없지만, `requires_grad=True` 가 설정되어 학습 가능한 매개변수를 갖는 모델을
평가(evaluate)할 때 유용합니다.

Autograd 구현에서 매우 중요한 클래스가 하나 더 있는데, 이것은 바로 ``Function``
클래스입니다.

``Tensor`` 와 ``Function`` 은 서로 연결되어 있으며, 모든 연산 과정을
부호화(encode)하여 순환하지 않는 그래프(acyclic graph)를 생성합니다. 각 tensor는
``.grad_fn`` 속성을 갖고 있는데, 이는 ``Tensor`` 를 생성한 ``Function`` 을
참조하고 있습니다. (단, 사용자가 만든 Tensor는 예외로, 이 때 ``grad_fn`` 은
``None`` 입니다.)

도함수를 계산하기 위해서는 ``Tensor`` 의 ``.backward()`` 를 호출하면
됩니다. 만약 ``Tensor`` 가 스칼라(scalar)인 경우(예. 하나의 요소 값만 갖는 등)에는
``backward`` 에 인자를 정해줄 필요가 없습니다. 하지만 여러 개의 요소를 갖고 있을
때는 tensor의 모양을 ``gradient`` 의 인자로 지정할 필요가 있습니다.



In [24]:
import torch
import numpy as np

tensor를 생성하고 ``requires_grad=True`` 를 설정하여 연산을 기록합니다.



In [3]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


tensor에 연산을 수행합니다:



In [4]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


``y`` 는 연산의 결과로 생성된 것이므로 ``grad_fn`` 을 갖습니다.



In [5]:
print(y.grad_fn)

``y`` 에 다른 연산을 수행합니다.



In [6]:
z = y * y * 3
out = z.mean()

print(z, out)

(tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>), tensor(27., grad_fn=<MeanBackward0>))


``.requires_grad_( ... )`` 는 기존 Tensor의 ``requires_grad`` 값을 바꿔치기
(in-place)하여 변경합니다. 입력값이 지정되지 않으면 기본값은 ``False`` 입니다.



In [7]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


변화도(Gradient)
-----------------
이제 역전파(backprop)를 해보겠습니다.
``out`` 은 하나의 스칼라 값만 갖고 있기 때문에, ``out.backward()`` 는
``out.backward(torch.tensor(1.))`` 과 동일합니다.



In [8]:
out.backward()

변화도 d(out)/dx를 출력합니다.




In [9]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


``4.5`` 로 이루어진 행렬을 확인할 수 있습니다. ``out`` 을 *Tensor* “$o$”
라고 하면, 다음과 같이 구할 수 있습니다.
$o = \frac{1}{4}\sum_i z_i$ 이고,
$z_i = 3(x_i+2)^2$ 이므로 $z_i\bigr\rvert_{x_i=1} = 27$ 입니다.
따라서,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$ 이므로,
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$ 입니다.



수학적으로 벡터 함수 $\vec{y}=f(\vec{x})$ 에서 $\vec{x}$ 에
대한 $\vec{y}$ 의 변화도는 야코비안 행렬(Jacobian Matrix)입니다:

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

일반적으로, ``torch.autograd`` 는 벡터-야코비안 곱을 계산하는 엔진입니다. 즉,
어떤 벡터 $v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$
에 대해 $v^{T}\cdot J$ 을 연산합니다. 만약 $v$ 가 스칼라 함수
$l=g\left(\vec{y}\right)$ 의 기울기인 경우,
$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$
이며, 연쇄법칙(chain rule)에 따라 벡터-야코비안 곱은 $\vec{x}$ 에 대한
$l$ 의 기울기가 됩니다:

\begin{align}J^{T}\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

(여기서 $v^{T}\cdot J$ 은 $J^{T}\cdot v$ 를 취했을 때의 열 벡터로
취급할 수 있는 행 벡터를 갖습니다.)

벡터-야코비안 곱의 이러한 특성은 스칼라가 아닌 출력을 갖는 모델에 외부 변화도를
제공(feed)하는 것을 매우 편리하게 해줍니다.



이제 벡터-야코비안 곱의 예제를 살펴보도록 하겠습니다:



In [21]:
x = torch.randn(3, requires_grad=True)
print(x)
y = x * 2
count = 1
# y.data.norm() = torch.sqrt(torch.sum(torch.pow(y, 2)))
while y.data.norm() < 1000:
    # print(y)
    count += 1
    y = y * 2
print(count)
print(y/x)
# print(x.grad)

tensor([-0.1352, -0.4725, -0.0398], requires_grad=True)
11
tensor([2048., 2048., 2048.], grad_fn=<DivBackward0>)


이 경우 ``y`` 는 더 이상 스칼라 값이 아닙니다. ``torch.autograd`` 는
전체 야코비안을 직접 계산할수는 없지만, 벡터-야코비안 곱은 간단히
``backward`` 에 해당 벡터를 인자로 제공하여 얻을 수 있습니다:



In [22]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)
# dydx = 1024!!!!!
print(v.mul(1024))
print(y.grad)
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])
None
tensor([2.0480e+02, 2.0480e+03, 2.0480e-01])


또한 ``with torch.no_grad():`` 로 코드 블럭을 감싸서 autograd가
``.requires_grad=True`` 인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있습니다.



In [23]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


**더 읽을거리:**

``autograd.Function`` 관련 문서는 https://pytorch.org/docs/stable/autograd.html#function
에서 찾아볼 수 있습니다.



In [39]:
t = np.array([0,1,2,3,4,5,6])
print(t)

print(t.ndim)
print(t.shape)

print(t[0],t[1],t[2])
print(t[2:5],t[4:-1])
print(t[:2],t[:3])

[0 1 2 3 4 5 6]
1
(7,)
(0, 1, 2)
(array([2, 3, 4]), array([4, 5]))
(array([0, 1]), array([0, 1, 2]))


In [40]:
t = np.array([[1,2,3],[4,5,6],[6,7,8],[1,2,3]])
print(t)

[[1 2 3]
 [4 5 6]
 [6 7 8]
 [1 2 3]]


In [41]:
t.ndim

2

In [43]:
t = torch.FloatTensor([0,1,2,3,4,5,6])
t

tensor([0., 1., 2., 3., 4., 5., 6.])

In [45]:
print(t.dim())
print(t.shape)
print(t.size())
print(t[0],t[2:5],t[:2])

1
torch.Size([7])
torch.Size([7])
(tensor(0.), tensor([2., 3., 4.]), tensor([0., 1.]))


In [47]:
m1 = torch.FloatTensor([[3,3]])
m2 = torch.FloatTensor([[2,2]])
print(m1+m2)
# 두 텐서간의 크기가 같지않아도 됨!

tensor([[5., 5.]])


In [48]:
m1 = torch.FloatTensor([[3]])
m2 = torch.FloatTensor([[2,2]])
print(m1+m2)
# 두 텐서간의 크기가 같지않아도 됨!

tensor([[5., 5.]])


In [50]:
m1 = torch.FloatTensor([[3,3]])
m2 = torch.FloatTensor([[2],[3]])
print(m1+m2)
# 두 텐서간의 크기가 같지않아도 됨!

tensor([[5., 5.],
        [6., 6.]])


In [52]:
m1 = torch.FloatTensor([[3,4],[1,2]])
m2 = torch.FloatTensor([[2],[3]])
print(m1.shape, m2.shape)
print(m1.matmul(m2)) # matmul은 흔히 생각하는 행렬곱
print(m1.mul(m2)) # mul은 같은 요소끼리만 곱함

(torch.Size([2, 2]), torch.Size([2, 1]))
tensor([[18.],
        [ 8.]])
tensor([[6., 8.],
        [3., 6.]])


In [54]:
t= torch.FloatTensor([1,2])
print(t.mean())
t = torch.LongTensor([1,2])
try:
    print(t.mean())
except Exception as exc:
    print(exc)

tensor(1.5000)
Can only calculate the mean of floating types. Got Long instead.


In [55]:
t = torch.FloatTensor([[1,2],[3,4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [58]:
print(t.mean())
print(t.mean(dim=0))
print(t.mean(dim=1))
print(t.mean(dim=-1)) 

tensor(2.5000)
tensor([2., 3.])
tensor([1.5000, 3.5000])
tensor([1.5000, 3.5000])


In [59]:
t = torch.FloatTensor([[1,2],[3,4]])
print(t)

tensor([[1., 2.],
        [3., 4.]])


In [60]:
print(t.max())

tensor(4.)


In [61]:
print(t.max(dim=0))
# argmax를 리턴 - 큰 원소에 대한 인덱스

torch.return_types.max(
values=tensor([3., 4.]),
indices=tensor([1, 1]))


In [62]:
print(t.max(dim=0)[0])

tensor([3., 4.])


In [76]:
t = np.array([[[[0,1,2],
              [0,1,2]]],
             [[0,1,2],
             [0,1,2]]])

# ft = torch.FloatTensor(t)
# print(ft.shape)

In [78]:
ft = torch.FloatTensor([[0],[1],[2]])
print(ft)
print(ft.shape)

tensor([[0.],
        [1.],
        [2.]])
torch.Size([3, 1])


In [83]:
print(ft.squeeze())
print(ft.squeeze().shape)
# dimension의 element 개수가 1인 경우에 그 dimension을 없애줌
x = ft.squeeze()
print(x)


tensor([0., 1., 2.])
torch.Size([3])
tensor([0., 1., 2.])


In [84]:
ft = torch.Tensor([0,1,2])
print(ft.shape)

torch.Size([3])


In [89]:
print(ft.unsqueeze(0))
print(ft.unsqueeze(1))
print(ft.view(1,-1))

tensor([[0., 1., 2.]])
tensor([[0.],
        [1.],
        [2.]])
tensor([[0., 1., 2.]])


In [91]:
print(ft.unsqueeze(-1))

tensor([[0.],
        [1.],
        [2.]])


In [92]:
lt = torch.LongTensor([1,2,3,4])
print(lt)

tensor([1, 2, 3, 4])


In [93]:
print(lt.float())

tensor([1., 2., 3., 4.])


In [95]:
bt = torch.ByteTensor([True,False , True, False])
print(bt)

tensor([1, 0, 1, 0], dtype=torch.uint8)


In [98]:
x = torch.LongTensor([[1,2],[3,4]])
y = torch.LongTensor([[1,2],[3,4]])

print(torch.cat([x,y],dim=0).shape)
print(torch.cat([x,y],dim=1).shape)

torch.Size([4, 2])
torch.Size([2, 4])


In [106]:
x = torch.FloatTensor([1,2,3,4])
y = torch.FloatTensor([1,2,3,4])
z = torch.FloatTensor([1,2,3,4])
print(torch.stack([x,y,z],dim=1))



tensor([[1., 1., 1.],
        [2., 2., 2.],
        [3., 3., 3.],
        [4., 4., 4.]])


In [107]:
x = torch.FloatTensor([[0,1,2],[2,1,0]])
print(x)

tensor([[0., 1., 2.],
        [2., 1., 0.]])


In [108]:
print(torch.ones_like(x))
print(torch.zeros_like(x))

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [109]:
x = torch.FloatTensor([[1,2],[3,4]])
print(x.mul(2.))
print(x)
print(x.mul_(2.))
# 원래의 x의 메모리에 x2를 해줌
print(x)

tensor([[2., 4.],
        [6., 8.]])
tensor([[1., 2.],
        [3., 4.]])
tensor([[2., 4.],
        [6., 8.]])
tensor([[2., 4.],
        [6., 8.]])


In [110]:
x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[4],[5],[6]])

In [111]:
W = torch.zeros(1,requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# requires_grad = True, 학습할 것이라고 명시
# W는 하나의 행렬, x 라는 input 벡터에 곱함
hypo = x_train * W + b

In [112]:
cost = torch.mean((hypo - y_train)**2)

In [115]:
import torch.optim as optim


optimizer = optim.SGD([W,b] , lr = 0.01)
# W,b라는 학습 가능한 변수
nb_epochs = 1000
for epoch in range(1,nb_epochs +1):
    hypo = x_train * W + b
    cost = torch.mean((hypo - y_train)**2)
    optimizer.zero_grad()
    # gradient 초기화
    cost.backward()
    # backward() 로 gradient 계산
    optimizer.step()
    # step()으로 계산된 gradient를 방향대로 Weight와 Bias, W와 b를 계산

In [120]:
import torch.optim as optim

x_train = torch.FloatTensor([[1],[2],[3]])
y_train = torch.FloatTensor([[4],[5],[6]])

W = torch.zeros(1)
lr = 0.1
nb_epochs = 10

for epoch in range(nb_epochs +1):
    hypo = x_train * W
    cost = torch.mean((hypo - y_train)**2)
    gradient = torch.sum((W*x_train - y_train) * x_train)
    
    print('Epoch {:4d}/{} W: {: .3f}, Cost: {:.6f}'.format(
    epoch,nb_epochs,W.item(),cost.item()
    ))
    W -= lr*gradient


Epoch    0/10 W:  0.000, Cost: 25.666666
Epoch    1/10 W:  3.200, Cost: 5.186668
Epoch    2/10 W:  1.920, Cost: 1.909867
Epoch    3/10 W:  2.432, Cost: 1.385579
Epoch    4/10 W:  2.227, Cost: 1.301692
Epoch    5/10 W:  2.309, Cost: 1.288271
Epoch    6/10 W:  2.276, Cost: 1.286123
Epoch    7/10 W:  2.289, Cost: 1.285780
Epoch    8/10 W:  2.284, Cost: 1.285725
Epoch    9/10 W:  2.286, Cost: 1.285716
Epoch   10/10 W:  2.285, Cost: 1.285715


In [133]:
# Multivariate Linear Regression
x_train = torch.FloatTensor([[73,80,75],
                            [93,88,93],
                            [89,91,80],
                            [96,98,100],
                            [73,66,70],])
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])


In [135]:
import torch.optim as optim
import torch.nn.functional as F
# Hx = w1x1 + w2x2 + w3x3 + b

# model init
W = torch.zeros((3,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([W,b],lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    hypo = x_train.matmul(W) + b
    cost = torch.mean((hypo- y_train) ** 2)
    optimizer.zero_grad()
    cost.backward()
    print(W.grad)
    # print(cost.backward())
    optimizer.step()
    print('Epoch {:4d}/{} hypo: {}, Cost: {:.6f}'.format(
    epoch, nb_epochs , hypo.squeeze().detach(), cost.item()
    ))
        

tensor([[-29401.2012],
        [-29360.0000],
        [-29018.0000]])
Epoch    0/20 hypo: tensor([0., 0., 0., 0., 0.]), Cost: 29661.800781
tensor([[-16672.5527],
        [-16646.4238],
        [-16445.7031]])
Epoch    1/20 hypo: tensor([66.7178, 80.1701, 76.1025, 86.0194, 61.1565]), Cost: 9537.694336
tensor([[-9456.2861],
        [-9438.7012],
        [-9318.0801]])
Epoch    2/20 hypo: tensor([104.5421, 125.6208, 119.2478, 134.7862,  95.8280]), Cost: 3069.590088
tensor([[-5365.1626],
        [-5352.4219],
        [-5277.2134]])
Epoch    3/20 hypo: tensor([125.9858, 151.3882, 143.7087, 162.4333, 115.4844]), Cost: 990.670898
tensor([[-3045.7773],
        [-3035.7820],
        [-2986.3220]])
Epoch    4/20 hypo: tensor([138.1429, 165.9963, 157.5768, 178.1071, 126.6283]), Cost: 322.482056
tensor([[-1730.8439],
        [-1722.4053],
        [-1687.5450]])
Epoch    5/20 hypo: tensor([145.0350, 174.2780, 165.4395, 186.9928, 132.9461]), Cost: 107.717064
tensor([[-985.3658],
        [-977.8093],

In [154]:
# minibatch = 전체 데이터를 균일하게 나눠서 학습하자
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73,80,75],
                            [93,88,93],
                            [89,91,80],
                            [96,98,100],
                            [73,66,70]]
        
        self.y_data = [[152],[185],[180],[196],[142]]
    
    def __len__(self):
        return len(self.x_data)
    # 데이터 셋의 총 데이터수
    
    def __getitem__(self,idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x,y
    # 어떠한 인덱스 idx를 받았을 때, 그에 상응하는 입출력 데이터 반환
    

dataset = CustomDataset()



    


In [156]:


class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super(MultivariateLinearRegressionModel,self).__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self,x):
        return self.linear(x)
    
    
model = MultivariateLinearRegressionModel()
# linear model 정의

dataloader = DataLoader(
    dataset,
    batch_size=2,
    # minibatch 의 크기 통상적으로 2의 제곱수로 설정
    shuffle=True,
    # Epoch 마다 데이터셋을 섞어서, 데이터가 학습되는 순서를 바꾼다.
    )

nb_epochs = 20
for epoch in range(nb_epochs+1):
    for batch_idx , samples in enumerate(dataloader):
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)
        
        cost = F.mse_loss(prediction, y_train)
            
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Batch: {}/{}, Cost: {:.6f}'.format(
        epoch, nb_epochs , batch_idx+1, len(dataloader),cost.item()
        ))
    

Epoch    0/20 Batch: 1/3, Cost: 38978.574219
Epoch    0/20 Batch: 2/3, Cost: 48398.703125
Epoch    0/20 Batch: 3/3, Cost: 27590.240234
Epoch    1/20 Batch: 1/3, Cost: 45641.023438
Epoch    1/20 Batch: 2/3, Cost: 39788.000000
Epoch    1/20 Batch: 3/3, Cost: 31486.746094
Epoch    2/20 Batch: 1/3, Cost: 36200.945312
Epoch    2/20 Batch: 2/3, Cost: 49228.082031
Epoch    2/20 Batch: 3/3, Cost: 31486.746094
Epoch    3/20 Batch: 1/3, Cost: 48398.703125
Epoch    3/20 Batch: 2/3, Cost: 37030.320312
Epoch    3/20 Batch: 3/3, Cost: 31486.746094
Epoch    4/20 Batch: 1/3, Cost: 38978.574219
Epoch    4/20 Batch: 2/3, Cost: 39788.000000
Epoch    4/20 Batch: 3/3, Cost: 44811.648438
Epoch    5/20 Batch: 1/3, Cost: 41736.253906
Epoch    5/20 Batch: 2/3, Cost: 45641.023438
Epoch    5/20 Batch: 3/3, Cost: 27590.240234
Epoch    6/20 Batch: 1/3, Cost: 45641.023438
Epoch    6/20 Batch: 2/3, Cost: 29538.492188
Epoch    6/20 Batch: 3/3, Cost: 51985.761719
Epoch    7/20 Batch: 1/3, Cost: 37030.320312
Epoch    7

In [158]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1)

In [161]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2],]
y_data = [[0],[0],[0],[1],[1],[1],]

In [163]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [164]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

('e^1 equals: ', tensor([2.7183]))


In [165]:
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [172]:
print(hypo)
# 기본 값이 0.5인 이유 = 1 / 1+ e^0
print(hypo.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


In [170]:
print('1/(1 + e^(-1)) equals: ', torch.sigmoid(torch.FloatTensor([1])))

('1/(1 + e^(-1)) equals: ', tensor([0.7311]))


In [176]:
# loss 가 Binary Cross Entropy이다.
# 이 loss는 Pytorch에서 제공해줌
# losses = -((y_train) * torch.log(hypo) + \ 
#            (1-y_train) * torch.log(1-hypo))

losses = F.binary_cross_entropy(hypo,y_train)


In [177]:
print(losses)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)


In [178]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [186]:
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([W,b], lr=1)
nb_epochs = 1000
for epoch in range(nb_epochs+1):
    #hypo = 1 / (1 + torch.exp(-(x_train.matmul(W)+b)))
    hypo = torch.sigmoid(x_train.matmul(W)+b)
    cost = F.binary_cross_entropy(hypo,y_train)

    optimizer.zero_grad()
    cost.backward()
    # 연산에 사용되었던 gradient
    optimizer.step()
    # step을 수행하면 cost 값을 최소화 하는 방향으로 업데이트함
    
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs ,cost.item()
        )) 
    

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


In [190]:
prediction = hypo >= torch.FloatTensor([0.5])
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [191]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [206]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier,self).__init__()
        self.linear = nn.Linear(8,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        return self.sigmoid(self.linear(x))

In [207]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2],]
y_data = [[0],[0],[0],[1],[1],[1],]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

model = BinaryClassifier()


In [209]:
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    hypo = model(x_train)
    
    cost = F.binary_cross_entropy(hypo, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 ==0:
        prediction = hypo >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
        epoch, nb_epochs ,cost.item(), accuracy * 100,
        )) 
    

RuntimeError: size mismatch, m1: [6 x 2], m2: [8 x 1] at ../aten/src/TH/generic/THTensorMath.cpp:136